Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](https://nlp100.github.io/data/jawiki-country.json.gz)がある．
- 1行に1記事の情報がJSON形式で格納される
- 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
- ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [1]:
import json

with open('jawiki-country.json', encoding='utf-8') as f:
    for row in f.readlines():
        if (row_dict := json.loads(row.strip()))['title'] == 'イギリス':
            text = row_dict['text']

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [2]:
import re

category_rows = [
    row for row in text.split('\n') if re.search('\[\[Category:.*\]\]', row)
]

category_rows

['[[Category:イギリス|*]]',
 '[[Category:イギリス連邦加盟国]]',
 '[[Category:英連邦王国|*]]',
 '[[Category:G8加盟国]]',
 '[[Category:欧州連合加盟国|元]]',
 '[[Category:海洋国家]]',
 '[[Category:現存する君主国]]',
 '[[Category:島国]]',
 '[[Category:1801年に成立した国家・領域]]']

## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [3]:
import re

categories = [
    match.group('name') for row in text.split('\n')
    if (match := re.match('\[\[Category:(?P<name>[^|]+).*\]\]', row))
]

categories

['イギリス',
 'イギリス連邦加盟国',
 '英連邦王国',
 'G8加盟国',
 '欧州連合加盟国',
 '海洋国家',
 '現存する君主国',
 '島国',
 '1801年に成立した国家・領域']

## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [4]:
import re

sections = [
    match.group() for row in text.split('\n')
    if (match := re.match('^=+.*=+$', row))
]

for section in sections:
    name = re.match('=+(?P<name>[^=]+)=+', section).group('name')
    level = section.count('=') // 2 - 1
    print(f'level: {level} - section: {name}')

level: 1 - section: 国名
level: 1 - section: 歴史
level: 1 - section: 地理
level: 2 - section: 主要都市
level: 2 - section: 気候
level: 1 - section: 政治
level: 2 - section: 元首
level: 2 - section: 法
level: 2 - section: 内政
level: 2 - section: 地方行政区分
level: 2 - section: 外交・軍事
level: 1 - section: 経済
level: 2 - section: 鉱業
level: 2 - section: 農業
level: 2 - section: 貿易
level: 2 - section: 不動産
level: 2 - section: エネルギー政策
level: 2 - section: 通貨
level: 2 - section: 企業
level: 3 - section: 通信
level: 1 - section: 交通
level: 2 - section: 道路
level: 2 - section: 鉄道
level: 2 - section: 海運
level: 2 - section: 航空
level: 1 - section: 科学技術
level: 1 - section: 国民
level: 2 - section: 言語
level: 2 - section: 宗教
level: 2 - section: 婚姻
level: 2 - section: 移住
level: 2 - section: 教育
level: 2 - section: 医療
level: 1 - section: 文化
level: 2 - section: 食文化
level: 2 - section: 文学
level: 2 - section: 哲学
level: 2 - section: 音楽
level: 3 - section: ポピュラー音楽
level: 2 - section: 映画
level: 2 - section: コメディ
level: 2 - section: 国花
level: 2 -

## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [5]:
import re

files = [
    match.group('name') for row in text.split('\n')
    if (match := re.match('.*\[\[ファイル:(?P<name>[^|]+).*\]\].*', row))
]

files

['Royal Coat of Arms of the United Kingdom.svg',
 'United States Navy Band - God Save the Queen.ogg',
 'Descriptio Prime Tabulae Europae.jpg',
 "Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg",
 'London.bankofengland.arp.jpg',
 'Battle of Waterloo 1815.PNG',
 'Uk topo en.jpg',
 'BenNevis2005.jpg',
 'Population density UK 2011 census.png',
 'Birmingham Skyline from Edgbaston Cricket Ground crop.jpg',
 'Glasgow and the Clyde from the air (geograph 4665720).jpg',
 'Palace of Westminster, London - Feb 2007.jpg',
 'Scotland Parliament Holyrood.jpg',
 'Donald Trump and Theresa May (33998675310) (cropped).jpg',
 'Soldiers Trooping the Colour, 16th June 2007.jpg',
 'City of London skyline from London City Hall - Oct 2008.jpg',
 'Oil platform in the North SeaPros.jpg',
 'Eurostar at St Pancras Jan 2008.jpg',
 'Airbus A380-841 G-XLEB British Airways (10424102995).jpg',
 'UKpop.svg',
 'Anglospeak.svg',
 "Royal Aberdeen Children's Hospital.jpg",
 'CHANDOS3.jpg',
 'The Fabs.JPG',
 'Wembley Stadium, 

## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [6]:
rows = []
is_inside = False
for row in text.split('\n'):
    if is_inside is True:
        if row.startswith('}}'):
            break

        if row.startswith('|'):
            rows.append(row)
        else:
            rows[-1] += row

    if row.startswith('{{基礎情報'):
        is_inside = True

dictionary = {
    match.group('name'): match.group('value')
    for row in rows
    if (match := re.match('\|(?P<name>.*\S)\s*=\s*(?P<value>\S.*)', row))
}

dictionary

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': "[[女王陛下万歳|{{lang|en|God Save the Queen}}]]

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [7]:
dictionary = {
    name: re.sub("''+", '', value) for name, value in dictionary.items()
}

dictionary

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]

## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [8]:
for name, value in dictionary.copy().items():
    value = re.sub('\[\[ファイル:(?P<name>[^|]+).*\]\]', '\g<name>', value)
    value = re.sub('\[\[(?P<name>[^\|]+?)\]\]', '\g<name>', value)
    value = re.sub('\[\[.+?\|(?P<name>.+?)\]\]', '\g<name>', value)
    dictionary[name] = value

dictionary

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': 'Royal Coat of Arms of the United Kingdom.svg',
 '国章リンク': '（国章）',
 '標語': '{{lang|fr|Dieu et mon droit}}<br />（フランス語:神と我が権利）',
 '国歌': '{{lang|en|God Save the Queen}}{{en icon}}<br />神よ女王を護り賜え<br />{{center|United States Navy Band - God Save the Queen.ogg}}',
 

## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki API](http://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Imageinfo)を呼び出して，ファイル参照をURLに変換すればよい）

In [9]:
import requests

S = requests.Session()

URL = 'https://en.wikipedia.org/w/api.php'

PARAMS = {
    'action': 'query',
    'format': 'json',
    'prop': 'imageinfo',
    'titles': f'File:{dictionary["国旗画像"]}',
    'iiprop': 'url'
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

PAGES = DATA['query']['pages']

for v in PAGES.values():
    url = v['imageinfo'][0]['url']
    
print(url)

https://upload.wikimedia.org/wikipedia/en/a/ae/Flag_of_the_United_Kingdom.svg
